#### Lin's feedback ####
In general, I noticed a cell type specific pattern, attached slide 1 is a correlation matrix I calculated based on the normalised_u. U2OS cell lines are standing out from all the other cell lines.  From the genome browser view in slide 2, you can see the cell type specific peaks highlighted in blue. And we know HCT116 cell line has much lower expression in APOBEC3B, therefore it is not surprising that in HCT116, the central peak at APOBEC3B intronic enhancer (highlighted in yellow) is lower and less distal peaks are observed.  


I noticed that you are using “starts_by_mean_expression” to set up high and med cutoff, which is the mean value from all the experiments? And the further downsteam peak calling and motif identification is also based on this mean value? If that is the case, I think that might lower the statistics power in general. I would highly recommend to separate the transcripts by exp groups, and perform the downstream analysis independently. The priority will be U2OS_WT and DLD1_WT experiments, because these are two cell lines we chose for screening. And it also makes more sense to find out the correlation between MERA hit regions and enhancers/promoters defined by this assay in  U2OS_WT and DLD1_WT experiments. Following these two sets of wild type experiments, we can cross compare the peaks and motifs in U2OS vs DLD1 vs HCT116 WT cell lines, and figure out which peak is common, and which is cell type specific.





#### Rich's feedback ####

1.       Where are the promoters/enhancers (STARR-Seq peaks) as defined by this assay for each experiment? This will allow us to ask whether there is correlation between MERA hit regions and enhancers/promoters as defined by this assay.

2.       Do any of the regions have differential activity in different experiments (normalized for technical differences between experiments like total reads)? This is especially interesting given Lin’s experimental setup that includes:

    a.       Wild-type human cell lines with high native APOBEC3B expression (U2OS, DLD-1).  
    b.      These cell lines with a knockout in NFKB, which she has found to be one driver of APOBEC3B expression.  
    c.       Wild-type HCT116 cells which do not express high native APOBEC3B.  
    d.      HCT116 cells treated with Gemcitabine, which potently activates APOBEC3B expresion.  

3.       Can we identify motifs or collections thereof that are responsible for enhancer/promoter activity? This should be possible with ~30-bp resolution given the tiled deletion strategy.

 

In [ ]:
printx("HI")

In [ ]:
import z1_save_oligos, z2_save_jaspar
oligos,oligos_by_exp = z1_save_oligos.load_oligos()
z2_save_jaspar.save_jaspar()
jaspar = z2_save_jaspar.load_jaspar()

### MOTIFS RESPONSIBLE FOR PROMOTER ACTIVITY ###

### GO TERM ENRICHMENT OF PROMOTER MOTIFS ###

_Next, could you please further elaborate on what have you modified and how you get this wider initial motif list? [...] I noticed that you mentioned “To this point I have tried to work mainly with log ratios of oligo expression levels within experiments...”. So did you split the experiments up first and then perform peak calling and motif scanning? If so, could you please share with me a table of the peak regions (both high and med) with the absolute numbers of transcript umis observed / barcode in each experiment?_


I made two changes -- first off, since we're specifically interested in finding differences between celltypes, I figured that we should be initially as permissive as possible in selecting mutants to create regions of interest which would be used for motif discovery. 

**CANDIDATE ENHANCER SUBREGION DISCOVERY**

First off, I did start calling initial peaks in each sample separately. To do this, employed a simple normalization procedure filtering the roughly the top 5% of all unmutated oligos in each experiment into a pool of enriched oligos. Then, for each filtered oligo, I created a list of all mutants which repressed transcription, either (a) below the 95th percentile, or (b) below the wildtype by 1 stddev. Finally, to keep any mutant subregion as a candidate of interest I demanded that it appear in at least two samples, whether in two replicates of one experiment or two independent experiments. This filter had similar permissivity to the search we applied last time. Each of these mutants was associated with a 30bp mutation subregion which would be considered in a later step for motif discovery. 

**SUBREGION EXTENSION AND MOTIF DISCOVERY**  

I joined each 30bp subregion with 15bp of flanking sequence on either sideto create 60 bp regions which would be used for motif discovery. For each 60bp subregion, I tried a de-novo and a database-guided approach to motif discovery. I figured we probably wouldn't have enough information for de novo discovery and that seems to be the case . We may return to this later. I don't think it will be productive yet.

I performed pwm-based motif scoring using 547 motifs from the HOMER dataset For each motif, we would like to say whether a given enhancer has a significant hit for that motif. Additive scoring of log odds for a position weight matrix will be higher for longer motifs. To judge relative goodness-of-fit for motifs of different lengths, we need to determine the expected score and standard deviation of each motif in the background. From this, we will compute a zscore for each possible motif hit. From a background model of nucleotide content based upon the APOBEC region, we computed a log odds probability of each motif in each 60bp sequence. Whereas before, I took exactly one, top-scoring motif hit for each region, this time I set a threshold based the log-odds probability over the background. We chose an FDR-based threshold of .005, and accepted any motif hits within each subregion meeting that threshold

To create the more permissive list of motifs, I merged this with the previously identified motifs. To create the stricter list of motifs, I took its intersection with this set.



In [ ]:

#LOADS BIOLOGICAL MOTIFS AND SCANS ALL SUBREGIONS FOR OCCURENCES
from pyfaidx import Fasta
sequences_fa = Fasta('/cluster/bh0085/genomes/GRCh38.primary_assembly.genome.fa')
chrseq = str(sequences_fa["chr22"])
all_60bp_subregions = pd.DataFrame()
subregions = None
for k,g in oligos_by_exp.groupby(level = "exp"):
    print(k)
    wt_oligos = g.loc[g.mutant_num == 0] 

    
    enriched_mu = np.percentile(wt_oligos.exp_norm_mu,95)
    enriched_wt_oligos = wt_oligos.loc[wt_oligos.exp_norm_mu > enriched_mu]
    enriched_wt_starts = enriched_wt_oligos.starts.unique()
    
    g["wt_mu"] = g.starts.apply(lambda x: (g.loc[(g["mutant_num"] == 0) & (g["starts"] == x)].exp_norm_mu.min()))
    g_wt_enriched =  g.loc[g.starts.isin(enriched_wt_starts)]
    non_enriched_mutants = g_wt_enriched.loc[ 
        (((g_wt_enriched.exp_norm_mu <enriched_mu) & (g_wt_enriched.mutant_num > 0)) |
        (g_wt_enriched.exp_norm_mu < (g_wt_enriched.wt_mu - g_wt_enriched.exp_norm_mu.std()))) 
    ]
    
    motif_30bp_genome_start_proposals = pd.Series(non_enriched_mutants.mutant_start.unique()) + oligos_by_exp.gstart.min()
    motif_60bp_subregions = motif_30bp_genome_start_proposals.apply(lambda x: chrseq[x - 15 : x+45])
    if subregions is None:
        subregions = pd.DataFrame(motif_60bp_subregions.rename("seq")).assign(exp=k)
    else:
        subregions = pd.concat([subregions, pd.DataFrame(motif_60bp_subregions.rename("seq")).assign(exp=k)],ignore_index = True)

subregions = subregions.loc[subregions[["seq"]].join(subregions.seq.value_counts().rename("seq_count"),on="seq").seq_count > 8]

In [186]:
#SCORE MOTIF REGIONS TO FIND ALL HITS WITHIN EACH SUBREGION IDENTIFIED ABOVE
from Bio import Seq, Alphabet
regions60  = pd.DataFrame(pd.Series(subregions.seq.unique()).rename("seq")).reset_index().set_index("seq").apply(lambda x: Seq.Seq(x.name,alphabet=Alphabet.IUPAC.IUPACUnambiguousDNA()),axis=1)
pwm_r60_grid = jaspar.pssm.apply(lambda  pssm: regions60.apply(lambda x:  sorted(pssm.search(x),key=lambda x:-1*x[1])))
pwm_r60_scores_grid = pwm_r60_grid.unstack().apply(lambda x: 0 if len(x)==0 else x[0][1]).unstack().T
pwm_r60_indices_grid = pwm_r60_grid.unstack().apply(lambda x: 0 if len(x)==0 else x[0][0]).unstack().T

In [275]:
#FILTER MOTIF SELECTIONS ACCORDING TO FDR RATE .005
all_seq_motif_matches = (pwm_r60_scores_grid.T - jaspar.pssm_score_threshold + 1*jaspar.pssm_score_std).reset_index().melt(id_vars=["seq"])
seq_motif_matches = all_seq_motif_matches .loc[all_seq_motif_matches.value > 0]
r60_max_motif_ids = seq_motif_matches[["jaspar_id","seq"]].rename({"jaspar_id":"motif_id"}) #pd.DataFrame(pwm_r60_scores_grid.idxmax().rename("motif_id"))
r60_max_motif_ids.to_csv("../data/0711_motif_manyregions_1std.csv")


#FILTER MOTIF SELECTIONS ACCORDING TO FDR RATE .005
all_seq_motif_matches = (pwm_r60_scores_grid.T - jaspar.pssm_score_threshold + .5*jaspar.pssm_score_std).reset_index().melt(id_vars=["seq"])
seq_motif_matches = all_seq_motif_matches .loc[all_seq_motif_matches.value > 0]
r60_max_motif_ids = seq_motif_matches[["jaspar_id","seq"]].rename({"jaspar_id":"motif_id"}) #pd.DataFrame(pwm_r60_scores_grid.idxmax().rename("motif_id"))
r60_max_motif_ids.to_csv("../data/0711_motif_manyregions_.5std.csv")

#FILTER MOTIF SELECTIONS ACCORDING TO FDR RATE .005
all_seq_motif_matches = (pwm_r60_scores_grid.T - jaspar.pssm_score_threshold + 1.5*jaspar.pssm_score_std).reset_index().melt(id_vars=["seq"])
seq_motif_matches = all_seq_motif_matches .loc[all_seq_motif_matches.value > 0]
r60_max_motif_ids = seq_motif_matches[["jaspar_id","seq"]].rename({"jaspar_id":"motif_id"}) #pd.DataFrame(pwm_r60_scores_grid.idxmax().rename("motif_id"))
r60_max_motif_ids.to_csv("../data/0711_motif_manyregions_1.5std.csv")



In [204]:
import matplotlib.pyplot as plt
%matplotlib inline

In [271]:

#CREATE A DATAFRAME WITH MOTIF CLUSTERING INFORMATION FOR EACH MOTIF IN EACH 60BP REGION OF INTEREST
import re
rc = lambda x: "".join([{"A":"T","G":"C","C":"G","T":"A"}[l] for l in x][::-1])

r60_max_centroids = r60_max_motif_ids.jaspar_id.apply(lambda x: jaspar.km_centroid_jaspar_id.get(x)).rename("cluster_centroid")
r60_km_cluster_ids = r60_max_motif_ids.jaspar_id.apply(lambda x: jaspar.km_cluster_id.get(x)).rename("km_cluster_id")
r60_spec2_cluster_ids = r60_max_motif_ids.jaspar_id.apply(lambda x: jaspar.spec2_cluster_id.get(x)).rename("spec2_cluster_id")
r60_spec3_cluster_ids = r60_max_motif_ids.jaspar_id.apply(lambda x: jaspar.spec3_cluster_id.get(x)).rename("spec3_cluster_id")
r60_ms_cluster_ids = r60_max_motif_ids.jaspar_id.apply(lambda x: jaspar.ms_cluster_id.get(x)).rename("ms_cluster_id")
r60_motif_positions = r60_max_motif_ids.apply(lambda x: pwm_r60_indices_grid[x.seq].loc[x].values[0],axis=1).rename("position").astype(np.int32)
r60_motif_positions = pd.concat([r60_motif_positions,r60_max_motif_ids.seq],axis=1)

r60_motif_scores = r60_max_motif_ids.apply(lambda x: pwm_r60_scores_grid[x.seq].loc[x].values[0],axis=1).rename("score")
r60_motif_lens = r60_max_motif_ids.apply(lambda x: jaspar.loc[x]["len"].values[0],axis=1).rename("motif_len")
r60_motif_seqs = pd.DataFrame(r60_motif_positions.apply(lambda x: x.seq[int(x.position):int(x.position+r60_motif_lens.loc[x.name])]
                                               if x.position >= 0 
                                               else rc(x.seq[int(60+x.position):int(60+x.position+r60_motif_lens.loc[x.name])]),axis=1)\
    .rename("motif_actual_seq")).set_index(r60_motif_positions.index).motif_actual_seq
r60 = pd.concat([r60_max_motif_ids,r60_max_centroids,
             r60_km_cluster_ids,r60_ms_cluster_ids,r60_spec3_cluster_ids,r60_spec2_cluster_ids,
             r60_motif_positions[["position"]],
             r60_motif_scores,r60_motif_lens,r60_motif_seqs],axis =1)


match60_positions = r60.apply(lambda x: [e.start() for e in re.compile(x.seq).finditer(chrseq[oligos.gstart.min():oligos.gend.max()])][0],axis=1)
match_motif_positions = (match60_positions + (r60.position + ((r60.position < 0) * 60)))
r60["motif_starts"] = match_motif_positions
r60.to_csv("../out/0709_r60_2.csv")


/cluster/bh0085/anaconda27/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
/cluster/bh0085/anaconda27/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  del sys.path[0]
/cluster/bh0085/anaconda27/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .re

In [274]:
match60_positions = r60.apply(lambda x: [e.start() for e in re.compile(x.seq).finditer(chrseq[oligos.gstart.min():oligos.gend.max()])][0],axis=1)
match_motif_positions = (match60_positions + (r60.position + ((r60.position < 0) * 60)))
r60["motif_starts"] = match_motif_positions

motifs = r60.drop_duplicates(["motif_starts","jaspar_id"]).reset_index(drop=True)
readout = None

for k,multiindexed_oligos in oligos_by_exp.groupby(["exp_nm","rep"]):
    these_oligos = multiindexed_oligos.reset_index(level=0)
    #these_oligos = oligos
    overlapping_oligo_starts = motifs.apply(lambda x: these_oligos.loc[(these_oligos.starts < x.motif_starts) & (these_oligos.starts > x.motif_starts - 120)].starts.unique(),axis =1)
    overlapping_wt_oligos = overlapping_oligo_starts.apply(lambda x: these_oligos.loc[these_oligos.starts.isin(x) & (these_oligos.mutant_num == 0)].RefSeqID)
    overlapping_mutant_oligos = motifs.apply(lambda x:these_oligos.loc[(these_oligos.mutant_start < (x.motif_starts+30)) & (these_oligos.mutant_start> (x.motif_starts - 30 ))].RefSeqID,axis=1)
    motif_overlap_wt_refseqs = overlapping_wt_oligos.stack()
    motif_overlap_mutant_refseqs = overlapping_mutant_oligos.stack()
    
    motif_mutant_mu = motif_overlap_mutant_refseqs.groupby(level=0).apply(lambda x:these_oligos.set_index("RefSeqID").loc[x].mu.mean()).rename("mutant_mu")
    motif_wt_mu = motif_overlap_wt_refseqs.groupby(level=0).apply(lambda x:these_oligos.set_index("RefSeqID").loc[x].mu.mean()).rename("wt_mu")
    motif_mutant_mu.index.name = "motif_hit_id"
    motif_wt_mu.index.name = "motif_hit_id"
    motifs.index.name = "motif_hit_id"
    
    muvals =  pd.concat([motif_mutant_mu,motif_wt_mu],axis=1)
    muvals["exp_nm"]=k[0]
    muvals["rep"]=k[1]
    if readout is None:
        readout = muvals
    else:
        readout = pd.concat([readout,muvals])

readout = readout.join(motifs[["km_cluster_id","ms_cluster_id","spec2_cluster_id","spec3_cluster_id"]],on="motif")
readout["rep"] = readout.rep.astype(np.int32)
melted = readout.reset_index().melt(value_name= "mu_val",var_name="mu_type",value_vars = ["mutant_mu","wt_mu"],id_vars=["exp_nm","rep","motif","ms_cluster_id","spec2_cluster_id","spec3_cluster_id","km_cluster_id"])


KeyError: 'motif'

### GO TERM ANALYSIS ###

In [ ]:
import numpy as np
from goatools.go_enrichment import GOEnrichmentStudy
from goatools import obo_parser
import wget
import os, pandas as pd
import os

#THIS IS A WAY TO PARSE THE FULL GO TREE
go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = '/cluster/bh0085/data/go'

# Check if we have the ./data directory already
if(not os.path.isfile(data_folder)):
    # Emulate mkdir -p (no error if folder exists)
    try:
        os.makedirs(data_folder)
    except OSError as e:
        if(e.errno != 17):
            raise e
else:
    raise Exception('Data path (' + data_folder + ') exists as a file. '
                   'Please rename, remove or change the desired location of the data path.')

# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'

# Import the OBO parser from GOATools
go = obo_parser.GODag(go_obo)



# THIS IS A WAY TO PARSE THE GO RECORDS DIRECTLY INTO A PANDAS DATAFRAME

from Bio.UniProt import GOA
fopen = open("/cluster/bh0085/data/go/goa_human.gaf")
itr = GOA.gafiterator(fopen)
records = list(itr)
ontologies = pd.DataFrame.from_dict(records)   

go_matches = jaspar.apply(lambda x: ontologies.loc[ontologies.DB_Object_Symbol.str.upper()==(x["name"].upper())].GO_ID.T,axis=1).reset_index()
go_melted = go_matches.melt(id_vars="jaspar_id").dropna().reset_index(drop=True)
semantic_terms = go_melted.value.apply(lambda term:go[term].name +" "+\
                                " ".join([e.name for e in list(go[term].parents)]) +\
                                " ".join([e.name for e in list(go[term].children)]) if term in go else "")

In [ ]:
semantic_terms.loc[semantic_terms.str.contains("hormone")]